In [20]:
import pandas as pd

def calculate_fantasy_points(player: pd.core.series.Series):
    points = 0

    # Batting Points
    points += player['runs_scored']  # 1 point per run
    points += player['no_of_fours']  # 1 point per boundary
    points += player['no_of_sixes'] * 2  # 2 points per six

    # Bonus for milestones
    if player['runs_scored'] >= 100:
        points += 16 
    elif player['runs_scored'] >= 50:
        points += 8 
    elif player['runs_scored'] >= 30 and player['match_type'] in ['T20', 'IT20']:
        points += 4

    # Calculate duck penalty (Ask Harsh and Sohit)
    if player['match_type'] not in ['Test', 'MDM']: 
        if player['runs_scored'] == 0 and player['out'] == 'out':
            if player['match_type'] in ['IT20', 'ODM', 'ODI', 'T20']:
                points -= 2  # Duck penalty for ODI and T20
    # ignoring duck penalty in test matches!


    # Strike Rate Bonus/Penalty (applicable in limited-overs matches)
    if player['match_type'] in ['IT20', 'ODM', 'ODI', 'T20'] and player['balls_faced'] >= 10:
        strike_rate = (player['runs_scored'] / player['balls_faced']) * 100
        if player['match_type'] in ['T20', 'IT20']:
            if strike_rate > 170:
                points += 6
            elif 150 < strike_rate <= 170:
                points += 4
            elif 130 < strike_rate <= 150:
                points += 2
            elif 60 <= strike_rate < 70:
                points -= 2
            elif 50 <= strike_rate < 60:
                points -= 4
            elif strike_rate < 50:
                points -= 6
        elif player['match_type'] in ['ODI', 'ODM']:
            if strike_rate > 140:
                points += 6
            elif 120 < strike_rate <= 140:
                points += 4
            elif 100 < strike_rate <= 120:
                points += 2
            elif 40 <= strike_rate < 50:
                points -= 2
            elif 30 <= strike_rate < 40:
                points -= 4
            elif strike_rate < 30:
                points -= 6

    # Bowling Points
    points += player['wickets'] * 25  # 25 points per wicket
    # points += player['maidens'] * 12  # 12 points per maiden over

    # Bonus for wicket types
    if 'LBWs/Bowled' in player:
        points += player['LBWs/Bowled'] * 8  # 8 points per LBW/Bowled wicket

    # Bonus for wicket hauls
    if player['wickets'] >= 5:
        points += 16
    elif player['wickets'] == 4:
        points += 8
    elif player['wickets'] == 3:
        points += 4

    # Economy Rate Bonus/Penalty (applicable in limited-overs matches)
    if player['match_type'] in ['IT20', 'ODM', 'ODI', 'T20'] and player['balls_bowled'] / 6 >= 2:
        economy_rate = player['runs_conceded'] / (player['balls_bowled'] / 6)
        if player['match_type'] in ['T20', 'IT20']:
            if economy_rate < 5:
                points += 6
            elif 5 <= economy_rate < 6:
                points += 4
            elif 6 <= economy_rate < 7:
                points += 2
            elif 10 <= economy_rate < 11:
                points -= 2
            elif 11 <= economy_rate < 12:
                points -= 4
            elif economy_rate >= 12:
                points -= 6
        elif player['match_type'] in ['ODI', 'ODI']:
            if economy_rate < 2.5:
                points += 6
            elif 2.5 <= economy_rate < 3.5:
                points += 4
            elif 3.5 <= economy_rate < 4.5:
                points += 2
            elif 7 <= economy_rate < 8:
                points -= 2
            elif 8 <= economy_rate < 9:
                points -= 4
            elif economy_rate >= 9:
                points -= 6

    # Fielding Points
    points += player['no_of_catches'] * 8  # 8 points per catch
    points += player['stumpings'] * 12  # 12 points per stumping
    # avg of (6 + 12) / 2 points per direct hit run-out and points per thrower/catcher run-out
    points += player['runouts'] * ((6 + 12) // 2)

    return points


# match_wise_player_data_file = '../CSVs/Overall_MatchWise.csv'
# df = pd.read_csv(match_wise_player_data_file, low_memory=False)
# player_data = df.iloc[0]
# fantasy_points = calculate_fantasy_points(player_data)
# print(f"Fantasy Points: {fantasy_points}")
# print(type(player_data))
# player_data.head()

In [27]:
match_wise_player_data_file = '../CSVs/Overall_MatchWise.csv'
output_file = match_wise_player_data_file.split('.csv')[0] + "_fantasy_points.csv"
output_file = '../CSVs/fantasy_points/Overall_MatchWise_fantasy_points.csv'
print(output_file)

../CSVs/fantasy_points/Overall_MatchWise_fantasy_points.csv


In [28]:
df = pd.read_csv(match_wise_player_data_file, low_memory=False)
df['fantasy_points'] = df.apply(calculate_fantasy_points, axis=1)
df.to_csv(output_file)

In [29]:
fpdf = pd.read_csv(output_file, low_memory=False)
fpdf['date'] = pd.to_datetime(fpdf['date'])

fpdf = fpdf.sort_values(by=['date'])

fpdf = fpdf.drop(columns='Unnamed: 0')
fpdf.to_csv(output_file, index=False)
fpdf

,match_id,player_id,player_name,runs_scored,balls_faced,no_of_fours,no_of_sixes,no_of_catches,runouts,balls_bowled,...,LBWs/Bowled,runs_conceded,stumpings,out,date,venue,match_type,gender,team_name,fantasy_points
388446,63963,ddc0828d,A Flintoff,0,4,0,0,0,0,168,...,2,50,0,out,2001-12-19,Bengaluru,Test,male,England,124
388447,63963,0c2730df,A Kumble,14,42,2,0,0,0,177,...,1,74,0,out,2001-12-19,Bengaluru,Test,male,India,49
388471,63963,de8cce37,VVS Laxman,12,18,2,0,1,0,0,...,0,0,0,out,2001-12-19,Bengaluru,Test,male,India,22
388470,63963,8ba8195d,V Sehwag,66,88,13,0,0,0,6,...,0,1,0,out,2001-12-19,Bengaluru,Test,male,India,87
388469,63963,b6cd714f,Sarandeep Singh,4,16,0,0,0,0,126,...,1,54,0,out,2001-12-19,Bengaluru,Test,male,India,120
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
324278,1451903,a77472dc,G Stewart,8,23,0,0,0,0,60,...,0,41,0,out,2024-11-13,Entebbe,ODM,male,Italy,83
324277,1451903,87e33d07,F Nsubuga,3,3,0,0,0,0,60,...,0,39,0,not out,2024-11-13,Entebbe,ODM,male,Uganda,3
326179,1456868,4d7c5b56,IK Gamantika,0,0,0,0,0,0,12,...,0,11,0,DNB,2024-11-13,Bali,T20,male,Indonesia,4
325324,1454819,d7bfcc0b,Mehran Khan,4,4,0,0,0,0,18,...,0,15,0,out,2024-11-13,Al Amarat,T20,male,Oman,33
